In [ ]:
'''
Plot 1D and 2D distribution of relevant variables to investigate correlations

'''

import awkward as ak
import uproot
import numpy as np
from scipy.optimize import curve_fit


import sys
import datetime
import numpy as np
import pandas as pd
import csv
import os
import subprocess
import ROOT as rt
import itertools


REPO_BASE_PATH = subprocess.Popen(['git', 'rev-parse', '--show-toplevel'], stdout=subprocess.PIPE).communicate()[0].rstrip().decode('utf-8')
plot_path = f"{REPO_BASE_PATH}/plots/ADR/time_resolution_202503/"
os.makedirs(plot_path, exist_ok=True)

sys.path.append(REPO_BASE_PATH + "/lib")
from histo_utilities import *

import tdrstyle
a = tdrstyle.setTDRStyle()

In [ ]:
files = {}
files_raw = {}
tree = {}
tree_raw = {}
# Open the input ROOT file
voltages = np.round([0.160],3)
biases = [4,1,2] #slot number of voltage source
pixels = [1,2,3] # pixel number in SNSPD
scope_ch = [2,3,4] # channel on scope


inputDirBase = "root://cmseos.fnal.gov///eos/uscms/store/user/christiw/SNSPD_data/ADR_time_resolution_202503/"
tempToDir = {
0.2: "20250311_0p2K",
# 0.5: "20250310_0p5K",
# 0.8: "20250309_0p8K",
# 1:"20250310_1p0K",

 }
for k, path in tempToDir.items():
    inputDir = f"{inputDirBase}/{path}/reco/v2/"
    for ch_i, bias_ch in enumerate(biases): # loop over pixels
        name = f"B{bias_ch}_P{pixels[ch_i]}"
        for j, v in enumerate(voltages): # loop over voltages  
            bv_string = str(voltages[j]).replace(".","p")
            files[f'{k}_{name}_{bv_string}'] = inputDir+f'output_run{name}_BV{bv_string}.root'
            
for k, v in files.items():
    if not os.path.exists(v[v.find("/eos"):]): 
        print("FILE NOT FOUND", v)
        continue
#     else: print(v)
    tree[k] = uproot.open(v)["pulse"]


In [ ]:
time = {}
rise_time = {}
amplitude = {}
time_nofit = {}
slope = {}
intercept = {}
for k, T in tree.items():
    amplitude[k] = T['amplitude'].array()
    rise_time[k] = T['rise_time'].array()
    time[k] = T['time'].array()
    time_nofit[k] = T['time_nofit'].array()
    slope[k] =  T['slope'].array()
    intercept[k] =  T['intercept'].array()

In [ ]:
# plot DCR for different channels in one canvas


c = rt.TCanvas("c1","gerrors2",200,10,700,500);
h = {}
leg = rt.TLegend(0.25,0.65,0.40,0.85)
leg.SetTextSize(0.05)
leg.SetTextFont(42)
leg.SetBorderSize(0)
leg.SetEntrySeparation(0.01)
leg.SetFillColor(0)


for k in tree.keys():
    if not "0.2_B4_P1_0p16" in k:continue
        
# #     h[k] = create_TH2D(np.column_stack((slope[k][:,1], time[k][:,1]-time[k][:,0],)), \
# #                        axis_title = ['SNSPD slope','SNSPD-laser time [ns]','Events'], binning = [100,0.05,0.25, 100,53,60])
#     h[k] = create_TH2D(np.column_stack((slope[k][:,0], time[k][:,1]-time[k][:,0],)), \
#                         axis_title = ['laser slope','SNSPD-laser time [ns]','Events'], binning = [100,0.6,1, 100,53,60])
#     h[k] = create_TH2D(np.column_stack((intercept[k][:,0], time[k][:,1]-time[k][:,0],)), \
#                         axis_title = ['laser intercept','SNSPD-laser time [ns]','Events'], binning = [100,40,60, 100,53,60])
# #     h[k] = create_TH2D(np.column_stack((slope[k][:,1], rise_time[k][:,1],)), \
# #                        axis_title = ['SNSPD slope','SNSPD rise time [ns]','Events'], binning = [100,0.0,0.4, 100,0,10])
#     h[k] = create_TH2D(np.column_stack((rise_time[k][:,1], time[k][:,1]-time[k][:,0],)), \
#                        axis_title = ['SNSPD rise time [ns]','SNSPD-laser time [ns]','Events'], binning = [100,2,8, 100,53,60])
# #     h[k] = create_TH2D(np.column_stack((intercept[k][:,1], time[k][:,1]-time[k][:,0],)), \
# #                        axis_title = ['SNSPD intercept','SNSPD-laser time [ns]','Events'], binning = [50,0.4,0.7, 100,53,60])
    h[k] = create_TH2D(np.column_stack((amplitude[k][:,1], time[k][:,1]-time[k][:,0],)), \
                       axis_title = ['SNSPD amplitude','SNSPD-laser time [ns]','Events'], binning = [50,0.08,0.14, 100,53,60])
#     h[k] = create_TH2D(np.column_stack((slope[k][:,1], time_nofit[k][:,1]-time_nofit[k][:,0],)), \
#                        axis_title = ['SNSPD slope','SNSPD-laser time nofit [ns]','Events'], binning = [100,0.05,0.25, 100,53,60])
#     h[k] = create_TH2D(np.column_stack((rise_time[k][:,1], time_nofit[k][:,1]-time_nofit[k][:,0],)), \
#                        axis_title = ['SNSPD rise time [ns]','SNSPD-laser time nofit [ns]','Events'], binning = [100,2,8, 100,53,60])
#     h[k] = create_TH2D(np.column_stack((intercept[k][:,1], time_nofit[k][:,1]-time_nofit[k][:,0],)), \
#                        axis_title = ['SNSPD intercept','SNSPD-laser time nofit [ns]','Events'], binning = [50,0.4,0.7, 100,53,60])

#     h[k] = create_TH2D(np.column_stack((time[k][:,1]-time[k][:,0], time_nofit[k][:,1]-time_nofit[k][:,0],)), \
#                        axis_title = ['SNSPD-laser time [ns]','SNSPD-laser time nofit [ns]','Events'], binning = [ 100,53,60, 100,53,60])
#     h[k] = create_TH2D(np.column_stack((time[k][:,1],time[k][:,0],)), \
#                        axis_title = ['SNSPD time [ns]','laser time [ns]','Events'], binning = [50,-2,2, 100,-60,-53])
#     h[k] = create_TH2D(np.column_stack((time_nofit[k][:,1],time[k][:,0],)), \
#                        axis_title = ['SNSPD time [ns]','laser time [ns]','Events'], binning = [50,-2,2, 100,-60,-53])

for i, k in enumerate(h.keys()):
    h[k].SetLineColor(std_color_list[i])
    h[k].SetMarkerColor(std_color_list[i])
    h[k].SetMarkerSize(0.9)
    h[k].SetMarkerStyle(8)
    h[k].SetTitle('')


    leg.AddEntry(h[k],k)
    h[k].Draw('hist')


c.Draw()
